In [ ]:
!pip install xgboost==1.6.1
!pip install scikit-optimize

In [ ]:
from xgboost import XGBRegressor
new_model=XGBRegressor()
new_model.load_model('/content/drive/MyDrive/Colab Notebooks/xgbr_avggc5in1110.h5')

In [ ]:
import pandas as pd
import numpy as np
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/average GC full dataset new 1110 for colab.csv')
Xtest=test.iloc[:,:5].values
Ytest = test.iloc[:,5:].values
print(Xtest[0])


In [ ]:
import numpy as np
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error


#define mape and mae
def mae(actual,pred):
     return (mean_absolute_error(actual,pred))
from sklearn.metrics import mean_absolute_percentage_error
def mape(actual,pred):
    calc=np.mean(np.absolute((actual-pred)*100/actual))
    return calc

In [ ]:
def prediction(input):
   
   pred=new_model.predict(input)
   preddf=pd.DataFrame(pred,index=None)
   predc2h4=preddf[2]
   predeth=preddf[4]
   predvalue=predc2h4+predeth
   predarr=predvalue.to_numpy()


   return predarr

def fitness(input):
    ans=prediction(input)
    return ans

In [ ]:
import random
#generate solutions
input=[]
for s in range(20):
    NPCtype=random.randint(1,2)
    
    NPCweight=random.randrange(50000,61000,100)/1000
    loading=random.randrange(29,49,1)/100
    KOHconc=random.choice([1,3,0.75])
    #ratio=random.randrange(7,11,1)
    current=random.randrange(100,1001,100)


    input.append([(NPCtype,NPCweight,loading,KOHconc,current)])

input=np.asarray(input)
mating_pool=input.reshape(input.shape[0],input.shape[2])

test=prediction(mating_pool)
test=np.asarray(test)

print(mating_pool)
print(test)

In [ ]:
#select first four as parents(best 4)

fitness_mating_pool=prediction(mating_pool)
mix_mating_fitness=np.column_stack((mating_pool,fitness_mating_pool))

mixsort=mix_mating_fitness[mix_mating_fitness[:, 5].argsort()[::-1]]
#[::-1] in argsort means to reverse the sorted results
parents_pool=mixsort[:,0:5]
parents_results=mixsort[:,5]
print(parents_pool)
print(parents_results)



In [ ]:
def parent_selection(mixsort):
    fitness_mating_pool=prediction(mating_pool)
    mix_mating_fitness=np.column_stack((mating_pool,fitness_mating_pool))

    mixsort=mix_mating_fitness[mix_mating_fitness[:, 5].argsort()[::-1]]
    parents_pool=mixsort[:,0:5]
    parents_results=mixsort[:,5]
    return parents_pool

print(parent_selection(mating_pool))

In [ ]:
parents_p2=parents_pool[:6,0:5]
print(parents_p2)

In [ ]:
# crossover(for mating)
def uniform_crossover(A,B,C,D,E,F,P):
    for i in range(len(P)):
        if P[i]<0.5:
          temp=A[i]
          A[i]=B[i]
          B[i]=C[i]
          C[i]=D[i]
          D[i]=E[i]
          E[i]=F[i]
          F[i]=temp
    return A,B,C,D,E,F

def crossover_process(mating_pool):
    A=mating_pool[0]
    B=mating_pool[1]
    C=mating_pool[2]
    D=mating_pool[3]
    E=mating_pool[4]
    F=mating_pool[5]
    P=np.random.rand(5)
    crossover_pool=uniform_crossover(A,B,C,D,E,F,P)
    crossover_pool=np.asarray(crossover_pool)
  
    return crossover_pool

cross_p2=crossover_process(parents_p2)
print(cross_p2)

In [ ]:
#mutation!
def mutation(crossover_pool):
    mutation_rate=random.uniform(0.8,1.2)
    mutation_X=crossover_pool[:,0]
    mutation_pool=crossover_pool[:,1:]*mutation_rate
    final_mutation=np.column_stack((mutation_X,mutation_pool))
    return final_mutation

mutated=mutation(cross_p2)
print(mutated)

In [ ]:
#survival selection by def
def survivor_selection():
    new_mating_pool=np.insert(mutation(crossover_process(parent_selection(mating_pool))),0,mating_pool,axis=0)
    fitness_new_pool=fitness(new_mating_pool)
    mix_new_fitness=np.column_stack((new_mating_pool,fitness_new_pool))

    mixsortnew=mix_new_fitness[mix_new_fitness[:, 5].argsort()]
    mixsortnew =mixsortnew[::-1]
    mixsortnew=mixsortnew[:]
    new_mating_pool=mixsortnew[0:20,0:5]
    return new_mating_pool
    


print(survivor_selection())
print(fitness(survivor_selection()))

In [ ]:
def genetic_algorithm():
    parents=parent_selection(mating_pool)
    
    
    cross_p=crossover_process(parents)
   
    mutant_p=mutation(cross_p)
    survived_p=survivor_selection()
    return survived_p
 
print(genetic_algorithm())  

In [ ]:
initial_gen=0
max_gen=5000

import numpy as np


while initial_gen<max_gen:
   x_new=genetic_algorithm()
   initial_gen+=1 
   mating_pool=x_new
   
   first=x_new[0]
   first=np.asarray(first)
   print([first])
   print("gen",initial_gen,"=",x_new)
   
   test=new_model.predict(x_new)
   c2test=test[:,2]+test[:,3]+test[:,4]
   print("best results:",test[0])
   
   print("Fittest C2H4+ethanol:",c2test[0])
   if c2test[0]>=70 : break
   elif np.sum(test[0])>=97.5: break

print(test[0],mating_pool[0])
print(new_model.predict(mating_pool))